In [3]:
import numpy as np
import pickle
from sklearn.feature_extraction import image
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt
%matplotlib inline

from train_model import train_step, test_step
from models.svm import DecisionTree
from utils.load_data import get_data
from utils.make_dict import train_bow, get_bow

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
args ={'dataset': 'cifar10',
       'dataroot': './data',
       'model': 'custom_SVM',
       'kernel': 'gaussian',
       'validation': 0.1,
       'C': 5.0,
       'sigma': 1.0,
       'batch': 1000,
       'dict_size': 100,
       'train': True,
       'load_cluster': False
       }

In [3]:
hyper_C = [0.5, 0.8, 0.9, 0.95, 1.0, 1.25]
hyper_sigma = [1.0, 1.25, 1.5, 2.0] 

In [2]:
trainX, trainy = get_data(dataset=args['dataset'], train=True, dataroot=args['dataroot'])
trainX = trainX.reshape((-1, 32, 32, 3), order='F')

if args['load_cluster']:
    with open("./cluster.dump", "rb") as f:
        cluster = pickle.load(f)
else:
    cluster = train_bow(trainX, num_dict=args['dict_size'], num_select=10000)
    with open("./cluster.dump", "wb") as f:
        pickle.dump(cluster, f)

trainFeature = get_bow(trainX, cluster, num_dict=args['dict_size'])

NameError: name 'get_data' is not defined

In [5]:
best_C = None
best_sigma = None
best_valid = 0.0

for C in hyper_C:
    for sigma in hyper_sigma:
        # Test hyperparameter
        args['C'] = C
        args['sigma'] = sigma

        # Get result
        _, train_acc_list, valid_acc_list  = \
            train_step(args, trainFeature, trainy)

        tra = sum(train_acc_list) / len(train_acc_list)
        val = sum(valid_acc_list) / len(valid_acc_list)

        if val > best_valid:
            best_valid = val
            best_C = C
            best_sigma = sigma

        # Print result
        print("C: %f Sigma: %f Train accuracy: %f Valid accuracy: %f"%(C, sigma, tra, val))

100%|██████████| 10/10 [00:08<00:00,  1.18it/s]


C: 0.500000 Sigma: 1.000000 Train accuracy: 100.000000 Valid accuracy: 43.072000


100%|██████████| 10/10 [00:07<00:00,  1.35it/s]


C: 0.500000 Sigma: 1.250000 Train accuracy: 100.000000 Valid accuracy: 34.956000


100%|██████████| 10/10 [00:07<00:00,  1.29it/s]


...

C: 1.250000 Sigma: 1.250000 Train accuracy: 100.000000 Valid accuracy: 34.786000


100%|██████████| 10/10 [00:07<00:00,  1.37it/s]


C: 1.250000 Sigma: 1.500000 Train accuracy: 100.000000 Valid accuracy: 59.078000


100%|██████████| 10/10 [00:07<00:00,  1.38it/s]

C: 1.250000 Sigma: 2.000000 Train accuracy: 100.000000 Valid accuracy: 34.892000


In [6]:
print("Best C: %f Best sigma: %f"%(best_C, best_sigma))

Best C: 0.950000 Best sigma: 1.250000


In [7]:
args['C'] = best_C
args['sigma'] = best_sigma
args['part'] = False
models, train_acc_list, valid_acc_list = \
            train_step(args, trainFeature)

100%|██████████| 10/10 [00:07<00:00,  1.41it/s]


In [8]:
testX, testy = get_data(dataset=args['dataset'], train=False, dataroot=args['dataroot'])
testX = testX.reshape((-1, 32, 32, 3), order='F')
testFeature = get_bow(testX, cluster, num_dict=args['dict_size'])

In [9]:
test_acc_list = test_step(args, testFeature, testy, models)

 90%|█████████ | 9/10 [00:07<00:00,  1.26it/s]


In [10]:
sum(test_acc_list) / len(test_acc_list)

65.68300000000002